In [37]:
from eth_utils import address
from web3 import Web3
import os
from solcx import compile_standard, install_solc
import json

version = '0.8.0'
install_solc(version)

smart_contract = "EnergyTrading.sol"

with open(smart_contract, "r") as file:
    contract_file = file.read()

compiled_sol = compile_standard(
    {
        "language": "Solidity",
        "sources": {"EnergyTrading.sol": {"content": contract_file}},
        "settings": {
            "outputSelection": {
                "*": {
                    "*": ["abi", "metadata", "evm.bytecode", "evm.bytecode.sourceMap"]
                }
            }
        },
    },
    solc_version=version,  # type: ignore
)

with open("compiled_code.json", "w") as file:
    json.dump(compiled_sol, file)

# get bytecode
bytecode = compiled_sol["contracts"][smart_contract]["EnergyTrading"]["evm"]["bytecode"]["object"]

# get abi
abi = json.loads(compiled_sol["contracts"][smart_contract]["EnergyTrading"]["metadata"])["output"]["abi"]

# set up connection
w3 = Web3(Web3.HTTPProvider("HTTP://172.21.128.1:7545"))
chain_id = 1337
owner_address = "0x952b1253dF851D217501Fa66fa5f4a8cC9025Eac"
private_key = "0c1aa4f476b3a2768675e3d1190ba26df71fd2e3044e01712db59538cdb82760"

# initialize contract
EnergyTrading = w3.eth.contract(abi=abi, bytecode=bytecode)
nonce = w3.eth.get_transaction_count(owner_address)

# set up transaction from constructor which executes when firstly
transaction = EnergyTrading.constructor().build_transaction(
    {
        "gasPrice": w3.eth.gas_price,
        "chainId": chain_id, 
        "from": owner_address, 
        "nonce": nonce
    }
)

signed_tx = w3.eth.account.sign_transaction(transaction, private_key=private_key)
tx_hash = w3.eth.send_raw_transaction(signed_tx.raw_transaction)
tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)

In [38]:
# REGISTER THE USER

energy_trading_sol = w3.eth.contract(abi=abi, address=tx_receipt.contractAddress)

payer_address = "0xB28d5f7929AA1F5f6d5811C692d59255ABFdD530"
private_key = "90c2213fbee613082e2340ed07de21e526d5b4a9b04dbc236780e799ee6bfa98"
nonce = w3.eth.get_transaction_count(payer_address)
value = 4

call_fun = energy_trading_sol.functions.registerUser().build_transaction(
    {
        "gasPrice": w3.eth.gas_price,
        "chainId": chain_id, 
        "from": payer_address,
        "nonce": nonce
    }
)

sign_call_fun = w3.eth.account.sign_transaction(call_fun, private_key=private_key)
tx_call_fun_hash = w3.eth.send_raw_transaction(sign_call_fun.raw_transaction)
tx_call_fun_receipt = w3.eth.wait_for_transaction_receipt(tx_call_fun_hash)

print(tx_call_fun_receipt)

AttributeDict({'transactionHash': HexBytes('0x16132da9c45317627a641dce0dc9ae61a9ac7f7ac86fcce4c06d5641bc9923db'), 'transactionIndex': 0, 'blockHash': HexBytes('0x7a7501c493533f1550b53ffaa2f697be543c975fa807185ad9c37c47bdedda8f'), 'blockNumber': 48, 'from': '0xB28d5f7929AA1F5f6d5811C692d59255ABFdD530', 'to': '0xfD8710ECD82e5dfa6b7CF816BD167B30f30d3286', 'gasUsed': 44415, 'cumulativeGasUsed': 44415, 'contractAddress': None, 'logs': [AttributeDict({'logIndex': 0, 'transactionIndex': 0, 'transactionHash': HexBytes('0x16132da9c45317627a641dce0dc9ae61a9ac7f7ac86fcce4c06d5641bc9923db'), 'blockHash': HexBytes('0x7a7501c493533f1550b53ffaa2f697be543c975fa807185ad9c37c47bdedda8f'), 'blockNumber': 48, 'address': '0xfD8710ECD82e5dfa6b7CF816BD167B30f30d3286', 'data': HexBytes('0x000000000000000000000000b28d5f7929aa1f5f6d5811c692d59255abfdd530'), 'topics': [HexBytes('0x54db7a5cb4735e1aac1f53db512d3390390bb6637bd30ad4bf9fc98667d9b9b9')], 'type': 'mined'})], 'status': 1, 'logsBloom': HexBytes('0x000000

In [39]:
# dump contract address to file
with open("../backend/utils/contract_address.json", "w") as file:
    json.dump({"contractAddress": tx_receipt.contractAddress}, file)

# dump contract abi to file
with open("../backend/utils/contract_abi.json", "w") as file:
    json.dump(abi, file)

In [63]:
# CREATE OFFER

energy_trading_sol = w3.eth.contract(abi=abi, address=tx_receipt.contractAddress)

payer_address = "0xB28d5f7929AA1F5f6d5811C692d59255ABFdD530"
private_key = "90c2213fbee613082e2340ed07de21e526d5b4a9b04dbc236780e799ee6bfa98"
nonce = w3.eth.get_transaction_count(payer_address)
value = 4

call_fun = energy_trading_sol.functions.offerEnergy(
    5000, 14, 500, 50
).build_transaction(
    {
        "gasPrice": w3.eth.gas_price,
        "chainId": chain_id, 
        "from": payer_address,
        "nonce": nonce
    }
)

sign_call_fun = w3.eth.account.sign_transaction(call_fun, private_key=private_key)
tx_call_fun_hash = w3.eth.send_raw_transaction(sign_call_fun.raw_transaction)
tx_call_fun_receipt = w3.eth.wait_for_transaction_receipt(tx_call_fun_hash)

print(tx_call_fun_receipt)

AttributeDict({'transactionHash': HexBytes('0xcaf584590ec35f03d3fdd2e3e1fec267833138eef1ac8bbf372291c15dc669d3'), 'transactionIndex': 0, 'blockHash': HexBytes('0xe38f39b87717621e7527b11eec2d868c3b3ff046c5be319104342b9cba224164'), 'blockNumber': 57, 'from': '0xB28d5f7929AA1F5f6d5811C692d59255ABFdD530', 'to': '0xfD8710ECD82e5dfa6b7CF816BD167B30f30d3286', 'gasUsed': 158377, 'cumulativeGasUsed': 158377, 'contractAddress': None, 'logs': [AttributeDict({'logIndex': 0, 'transactionIndex': 0, 'transactionHash': HexBytes('0xcaf584590ec35f03d3fdd2e3e1fec267833138eef1ac8bbf372291c15dc669d3'), 'blockHash': HexBytes('0xe38f39b87717621e7527b11eec2d868c3b3ff046c5be319104342b9cba224164'), 'blockNumber': 57, 'address': '0xfD8710ECD82e5dfa6b7CF816BD167B30f30d3286', 'data': HexBytes('0x0000000000000000000000000000000000000000000000000000000000000006000000000000000000000000b28d5f7929aa1f5f6d5811c692d59255abfdd5300000000000000000000000000000000000000000000000000000000000001388000000000000000000000000000000

In [64]:
# GET ACTIVE OFFERS

energy_trading_sol = w3.eth.contract(abi=abi, address=tx_receipt.contractAddress)
payer_address = "0xB28d5f7929AA1F5f6d5811C692d59255ABFdD530"
private_key = "90c2213fbee613082e2340ed07de21e526d5b4a9b04dbc236780e799ee6bfa98"
nonce = w3.eth.get_transaction_count(payer_address)
value = 4

# Call the function and get the return value
active_offers = energy_trading_sol.functions.getActiveOffers().call({
    "from": payer_address
})

active_offers

[(0,
  '0xB28d5f7929AA1F5f6d5811C692d59255ABFdD530',
  5000,
  14,
  1732197541,
  '0x22f5145d4C248227B2FbC5F90315B9aD62dEBCFa',
  2000,
  False),
 (1,
  '0xB28d5f7929AA1F5f6d5811C692d59255ABFdD530',
  5000,
  14,
  1732197542,
  '0x0000000000000000000000000000000000000000',
  500,
  False),
 (2,
  '0xB28d5f7929AA1F5f6d5811C692d59255ABFdD530',
  5000,
  14,
  1732197543,
  '0x0000000000000000000000000000000000000000',
  500,
  False),
 (3,
  '0xB28d5f7929AA1F5f6d5811C692d59255ABFdD530',
  5000,
  14,
  1732197544,
  '0x0000000000000000000000000000000000000000',
  500,
  False),
 (4,
  '0x22f5145d4C248227B2FbC5F90315B9aD62dEBCFa',
  5000,
  14,
  1732198261,
  '0x0000000000000000000000000000000000000000',
  500,
  False),
 (5,
  '0xB28d5f7929AA1F5f6d5811C692d59255ABFdD530',
  5000,
  14,
  1732198324,
  '0x0000000000000000000000000000000000000000',
  500,
  False)]

In [48]:
# REGISTER THE USER

energy_trading_sol = w3.eth.contract(abi=abi, address=tx_receipt.contractAddress)

payer_address = "0x22f5145d4C248227B2FbC5F90315B9aD62dEBCFa"
private_key = "f8615fbd141bf5cc9156f3b6e21f966bbd22cd2d530b069d6033cd929700afe1"
nonce = w3.eth.get_transaction_count(payer_address)
value = 4

call_fun = energy_trading_sol.functions.registerUser().build_transaction(
    {
        "gasPrice": w3.eth.gas_price,
        "chainId": chain_id, 
        "from": payer_address,
        "nonce": nonce
    }
)

sign_call_fun = w3.eth.account.sign_transaction(call_fun, private_key=private_key)
tx_call_fun_hash = w3.eth.send_raw_transaction(sign_call_fun.raw_transaction)
tx_call_fun_receipt = w3.eth.wait_for_transaction_receipt(tx_call_fun_hash)

print(tx_call_fun_receipt)

AttributeDict({'transactionHash': HexBytes('0x1b0b18fba7570ade90fcbbf80f445297d2379085604dda326c92d38f029b2db8'), 'transactionIndex': 0, 'blockHash': HexBytes('0x15d12c4e20ff7853be5348d0154e2c8db23413a8af130758f7001fb7dcb1bc1d'), 'blockNumber': 53, 'from': '0x22f5145d4C248227B2FbC5F90315B9aD62dEBCFa', 'to': '0xfD8710ECD82e5dfa6b7CF816BD167B30f30d3286', 'gasUsed': 44415, 'cumulativeGasUsed': 44415, 'contractAddress': None, 'logs': [AttributeDict({'logIndex': 0, 'transactionIndex': 0, 'transactionHash': HexBytes('0x1b0b18fba7570ade90fcbbf80f445297d2379085604dda326c92d38f029b2db8'), 'blockHash': HexBytes('0x15d12c4e20ff7853be5348d0154e2c8db23413a8af130758f7001fb7dcb1bc1d'), 'blockNumber': 53, 'address': '0xfD8710ECD82e5dfa6b7CF816BD167B30f30d3286', 'data': HexBytes('0x00000000000000000000000022f5145d4c248227b2fbc5f90315b9ad62debcfa'), 'topics': [HexBytes('0x54db7a5cb4735e1aac1f53db512d3390390bb6637bd30ad4bf9fc98667d9b9b9')], 'type': 'mined'})], 'status': 1, 'logsBloom': HexBytes('0x000000

In [69]:
energy_trading_sol = w3.eth.contract(abi=abi, address=tx_receipt.contractAddress)

payer_address = "0x22f5145d4C248227B2FbC5F90315B9aD62dEBCFa"
private_key = "f8615fbd141bf5cc9156f3b6e21f966bbd22cd2d530b069d6033cd929700afe1"
nonce = w3.eth.get_transaction_count(payer_address)
value = 4

call_fun = energy_trading_sol.functions.placeBid(
    0
).build_transaction(
    {
        "value": w3.to_wei("0.33", "ether"),
        "gasPrice": w3.eth.gas_price,
        "chainId": chain_id, 
        "from": payer_address,
        "nonce": nonce
    }
)

sign_call_fun = w3.eth.account.sign_transaction(call_fun, private_key=private_key)
tx_call_fun_hash = w3.eth.send_raw_transaction(sign_call_fun.raw_transaction)
tx_call_fun_receipt = w3.eth.wait_for_transaction_receipt(tx_call_fun_hash)

print(tx_call_fun_receipt)

AttributeDict({'transactionHash': HexBytes('0xacdddfaaa7acc1c754f44e519af01a031d4151ba53f8780d6cc0a0946d7669c9'), 'transactionIndex': 0, 'blockHash': HexBytes('0xaa6589617a2faa9d96b1686516b86d4abecd0c505856a3c14194b417dec1a40c'), 'blockNumber': 59, 'from': '0x22f5145d4C248227B2FbC5F90315B9aD62dEBCFa', 'to': '0xfD8710ECD82e5dfa6b7CF816BD167B30f30d3286', 'gasUsed': 94702, 'cumulativeGasUsed': 94702, 'contractAddress': None, 'logs': [AttributeDict({'logIndex': 0, 'transactionIndex': 0, 'transactionHash': HexBytes('0xacdddfaaa7acc1c754f44e519af01a031d4151ba53f8780d6cc0a0946d7669c9'), 'blockHash': HexBytes('0xaa6589617a2faa9d96b1686516b86d4abecd0c505856a3c14194b417dec1a40c'), 'blockNumber': 59, 'address': '0xfD8710ECD82e5dfa6b7CF816BD167B30f30d3286', 'data': HexBytes('0x00000000000000000000000022f5145d4c248227b2fbc5f90315b9ad62debcfa0000000000000000000000000000000000000000000000000494654067e10000'), 'topics': [HexBytes('0xdd0b6c6a77960e2066c96171b4d7ac9e8b4c184011f38544afa36a5bb63ec59f')], 

In [55]:
# GET ACTIVE OFFERS

energy_trading_sol = w3.eth.contract(abi=abi, address=tx_receipt.contractAddress)

payer_address = "0xB28d5f7929AA1F5f6d5811C692d59255ABFdD530"

# Call the function and get the return value
active_offers = energy_trading_sol.functions.getBidsByOffer(0).call({
    "from": payer_address
})

active_offers

[('0x22f5145d4C248227B2FbC5F90315B9aD62dEBCFa', 1000, 0),
 ('0x22f5145d4C248227B2FbC5F90315B9aD62dEBCFa', 2000, 0)]

In [65]:
# GET ACTIVE OFFERS

energy_trading_sol = w3.eth.contract(abi=abi, address=tx_receipt.contractAddress)

payer_address = "0xB28d5f7929AA1F5f6d5811C692d59255ABFdD530"

# Call the function and get the return value
active_offers = energy_trading_sol.functions.getOffersByUser(payer_address).call({
    "from": payer_address
})

active_offers

[(0,
  '0xB28d5f7929AA1F5f6d5811C692d59255ABFdD530',
  5000,
  14,
  1732197541,
  '0x22f5145d4C248227B2FbC5F90315B9aD62dEBCFa',
  2000,
  False),
 (1,
  '0xB28d5f7929AA1F5f6d5811C692d59255ABFdD530',
  5000,
  14,
  1732197542,
  '0x0000000000000000000000000000000000000000',
  500,
  False),
 (2,
  '0xB28d5f7929AA1F5f6d5811C692d59255ABFdD530',
  5000,
  14,
  1732197543,
  '0x0000000000000000000000000000000000000000',
  500,
  False),
 (3,
  '0xB28d5f7929AA1F5f6d5811C692d59255ABFdD530',
  5000,
  14,
  1732197544,
  '0x0000000000000000000000000000000000000000',
  500,
  False),
 (5,
  '0xB28d5f7929AA1F5f6d5811C692d59255ABFdD530',
  5000,
  14,
  1732198324,
  '0x0000000000000000000000000000000000000000',
  500,
  False)]

In [70]:
# get contract balance

w3.eth.get_balance(tx_receipt.contractAddress)

330000000000000000